## Clusterização e Análise Descritiva da pobreza no Brasil.

### Fonte dos dados: https://www.kaggle.com/patrickgomes/determinants-of-poverty-in-brazil
---

### Problema: Identificar possíveis clusters de indivíduos entre aqueles classificados como pobres e não-pobres. Depois da clusterização faremos a análise descritiva do que foi descoberto.

### Tipo do modelo: Não Supervisionado Offline do tipo Clusterizador.

### Modelos usados: KMeans e DBSCAN.

### Métrica de desempenho: Coeficiente de Silhueta.

## Bibliotecas

In [1]:
# Recursos matemáticos, manipulação e estruturação de dados.
import pandas as pd
import numpy as np

# Pré-processamento, métricas e algoritmos de clusterização.
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.cluster import KMeans, DBSCAN
from sklearn.metrics import silhouette_score

# Filtragem de avisos.
import warnings
warnings.filterwarnings('ignore')

# Semente de reprodução.
np.random.seed(0)

# Fazendo o download dos dados.
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Load, Overview e Pré-processamento

In [4]:
# Talvez você precise atualizar o pacote openpyxl usando pip:
#!pip install openpyxl --upgrade

dados = pd.read_excel('/content/drive/Shareddrives/IFSC/IML/P6.Clusterização/poverty_brazil.xlsx')

In [5]:
#woman: 1 - feminino 0 - masculino
#age: Idade da pessoa

#education: 1 - Sem escolaridade e menos de 1 ano de estudo 2 - Fundamental incompleto ou equivalente
#           3 - Fundamental completo ou equivalente 4 - Áudio incompleto ou equivalente
#           5 - Áudio completo ou equivalente 6 - Superior incompleto ou equivalente 7 - Superior completo

#work: 1 - Agricultura, pecuária, silvicultura, pesca e aquicultura 2 - Indústria em geral
#      3 - Construção 4 - Comércio, reparação de veículos automóveis e motociclos
#      5 - Transporte, armazenagem e correio 6 - Alojamento e alimentação
#      7 - Informação, comunicação e financeiro, imobiliário , profissional e administrativo
#      8 - Administração pública, defesa e segurança social 9 - Educação, saúde humana e serviços sociais
#      10 - Outros serviços 11 - Serviços domiciliários 12 - Actividades mal definidas

#metropolitan_area: 1 - mora em região metropolitana 0 - não mora em região metropolitana
#non_white: 1 - não branco (preto, pardo, amarelo e indígena) 0 - branco
#urban: 1 - vive na zona urbana 0 - vive no campo
#work_permit: 0 - não tem autorização de trabalho 1 - tem autorização de trabalho
#             2 - outras situações, empregador, funcionário público
#poverty: 1 - foi ruim na entrevista de 2020 0 - não foi ruim na entrevista de 2020

In [7]:
dados

,woman,age,education,work,metropolitan_area,non_white,urban,work_permit,poverty
0,0,59,5,8,1,1.0,1,2,0
1,0,21,5,8,1,0.0,1,0,0
2,0,59,2,9,1,1.0,1,1,0
3,1,58,5,8,1,1.0,1,2,0
4,1,56,2,9,1,1.0,1,1,0
...,...,...,...,...,...,...,...,...,...
20747,1,46,4,11,1,1.0,1,0,0
20748,0,47,1,3,1,0.0,1,2,0
20749,0,41,7,4,1,1.0,1,1,0
20750,1,42,7,7,1,1.0,1,1,0


#### Percebe-se que por mais que vários atributos sejam categóricos como 'education', 'work' e 'work permit', eles estão codificados em inteiro. A lista dos dados codificados pode ser encontrada no Kaggle.
---

In [ ]:
dados.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20752 entries, 0 to 20751
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   woman              20752 non-null  int64  
 1   age                20752 non-null  int64  
 2   education          20752 non-null  int64  
 3   work               20752 non-null  int64  
 4   metropolitan_area  20752 non-null  int64  
 5   non_white          20751 non-null  float64
 6   urban              20752 non-null  int64  
 7   work_permit        20752 non-null  int64  
 8   poverty            20752 non-null  int64  
dtypes: float64(1), int64(8)
memory usage: 1.4 MB


#### Como há apenas um valor nulo na coluna non_white, simplesmente o excluiremos.
---

In [8]:
dados.dropna(inplace=True)

In [9]:
dados.duplicated().sum()

5770

#### Há cerca de 5770 duplicatas (instâncias iguais). Não há um ID de identificação pessoal no banco de dados, mas vamos supor que são pessoas diferentes.

In [10]:
dados.woman.astype(str).value_counts()/dados.shape[0]

,count
woman,
0,0.606429
1,0.393571


#### Um fato importante e que mostra que os dados não são representativos da população é a distribuição entre homens (61%) e mulheres (39%). Em condições não enviesadas e puramente aleatórias, no Brasil, teríamos algo em torno de 50% de cada sexo. Com isso em mente, toda a análise que será feita é válida apenas para essa amostra de dados.
---

## Definindo os Clusters e Análise

#### Para encontrar o número ideal de clusters no modelo KMeans temos duas abordagens, a primeira é identificar o cotovelo (elbow) no gráfico do número de clusters pela inércia, e a segunda é avaliar o coeficiente de silhueta do modelo. A segunda abordagem é mais objetiva e é ela que usaremos. No modelo DBSCAN não precisamos definir o número de clusters já que o algoritmo os encontra baseado-se em regiões de alta densidade, delimitadas por ε, em torno de instâncias representativas, chamadas de core. Para avaliar um modelo DBSCAN, no entanto, também podemos usar a métrica da silhueta. A literatura¹ recomenda que os dados sejam escalonados antes de serem usados nos modelos de clusterização, e para avaliar diferentes conjuntos de dados (não escalonado e escalonado) criaremos uma função que facilitará a reutilização do código.

##### ¹ GÉRON, Aurélien. Mãos à Obra: Aprendizado de Máquina com Scikit-Learn & TensorFlow: Rio de Janeiro: Alta Books, 2021.

In [11]:
def melhor_cluster(dados, limite, escalonador=False):

    if escalonador:
        dados = escalonador().fit_transform(dados)
        print('\nEscalonador:', escalonador.__name__)
    else:
        print('\nEscalonador: Nenhum')

    # ------------------------- Encontrando o melhor modelo KMeans ------------------------- #

    scores_kmeans = []

    for n_clusters in range(2, limite+1):
        kmeans = KMeans(n_clusters=n_clusters)
        kmeans.fit(dados)
        scores_kmeans.append((n_clusters, silhouette_score(dados, kmeans.labels_)))

    scores_kmeans = np.asarray(scores_kmeans)
    n_clusters, coef = scores_kmeans[np.argmax(scores_kmeans[:, 1])]

    print(f'KMeans --> Clusters: {n_clusters} | Silhueta: {coef}')

    # -------------------------------------------------------------------------------------- #

    # ------------------------- Encontrando o melhor modelo DBSCAN ------------------------- #

    scores_dbscan = []

    # Recomenda-se que o número mínimo de instâncias ao redor de uma instância core seja igual
    # a pelo menos a quantidade de atributos.
    min_samples = dados.shape[1] + 1

    for i in np.linspace(0.1, 10, 100):
        dbscan = DBSCAN(eps=i, min_samples=min_samples)
        dbscan.fit(dados)

        # Conforme ε aumenta, maior é a tendência de formarmos um único cluster que englobará
        # todas as instâncias. Caso isso aconteça, teremos um erro no cálculo da silhueta (uma
        # vez que ela se baseia na distância intra e entre clusters) e interromperemos a iteração.
        try:
            scores_dbscan.append((i, silhouette_score(dados, dbscan.labels_)))
        except ValueError:
            break

    scores_dbscan = np.asarray(scores_dbscan)
    eps, coef = scores_dbscan[np.argmax(scores_dbscan[:, 1])]

    print(f'DBSCAN --> Epsilon (ε): {eps} | Silhueta: {coef}\n\n')

    # -------------------------------------------------------------------------------------- #

### Identificando Clusters de Pessoas Pobres

#### Aquelas atributos categóricos já citados não fazem sentido lógico quando criamos grupos de instâncias com dados nominais e codificados da forma como estão. Uma solução para usar esses dados seria usar uma matriz esparsa contendo a codificação OneHot desses atributos. Para ver aplicações práticas da codificação OneHot confira minha análise usando modelos de classificação no link:

https://www.kaggle.com/joaovitorsilva/classificador-de-acidentes-e-an-lise-de-2020

#### Não faremos esse tratamento nessa análise e simplesmente excluiremos tais atributos.

In [12]:
X_pobre = dados[dados.poverty == 1].drop(['education', 'work', 'work_permit', 'poverty'], axis=1)

#### Faremos três buscas de modelos, uma com dados não escalonados, outra com dados escalonados por padronização e a última com escalonamento por normalização.

In [13]:
# Se algum modelo KMeans encontrar o número de clusters ideal como sendo o argumento 'limite', então reiteraremos com um limite maior. Começaremos com 80.
for i in [False, StandardScaler, MinMaxScaler]:
    melhor_cluster(X_pobre, 80, i)


Escalonador: Nenhum
KMeans --> Clusters: 2.0 | Silhueta: 0.5581330834422994
DBSCAN --> Epsilon (ε): 4.2 | Silhueta: 0.6845790254778434



Escalonador: StandardScaler
KMeans --> Clusters: 68.0 | Silhueta: 0.5574054724022032
DBSCAN --> Epsilon (ε): 1.1 | Silhueta: 0.5447314520375169



Escalonador: MinMaxScaler
KMeans --> Clusters: 14.0 | Silhueta: 0.8151923980457658
DBSCAN --> Epsilon (ε): 0.4 | Silhueta: 0.8224213526130029




#### O modelo com melhor desempenho foi o DBSCAN com ε = 0.4 para os dados normalizados. Vamos então treinar e avaliar os clusters desse modelo.

In [15]:
def clusters(dados_original, modelo, escalonador=False):

    dados = dados_original.copy()

    if escalonador:
        # Normalizando os dados e os colocando num DataFrame.
        dados = pd.DataFrame(escalonador().fit_transform(dados), columns=dados.columns)

    # Treinando o modelo.
    modelo.fit(dados)

    # Adicionando a coluna dos clusters previstos.
    dados['cluster'] = modelo.labels_

    if escalonador:
        # Uma vez com o modelo treinado, podemos recuperar a coluna de idades para posterior análise.
        dados.age = dados_original.age.values

    # Agrupando por cluster e somando todas as instâncias.
    clusters = dados.groupby('cluster').sum()

    # Reindexando por clusters mais populosos em ordem decrescente.
    maiores_clusters_idx = dados.groupby('cluster').size().sort_values(ascending=False).index
    clusters = clusters.reindex(maiores_clusters_idx)

    # Fazendo a média de idade.
    clusters.age = (clusters.age/dados.groupby('cluster').size().sort_values(ascending=False)).values
    clusters.rename({'age': 'mean_age'}, axis=1, inplace=True)

    # Tamanho dos clusters.
    clusters['size'] = dados.groupby('cluster').size().sort_values(ascending=False)

    return clusters

In [17]:
dbscan = DBSCAN(eps=0.2)
clusters(X_pobre, dbscan, MinMaxScaler)

,woman,mean_age,metropolitan_area,non_white,urban,size
cluster,,,,,,
5,0.0,41.188492,0.0,1008.0,0.0,1008
2,0.0,39.424695,0.0,737.0,737.0,737
0,0.0,38.554545,440.0,440.0,440.0,440
3,439.0,39.838269,0.0,439.0,439.0,439
7,0.0,42.191646,0.0,0.0,0.0,407
6,387.0,40.304910,387.0,387.0,387.0,387
4,0.0,40.398601,0.0,0.0,286.0,286
9,235.0,38.357447,0.0,235.0,0.0,235
11,202.0,39.702970,0.0,0.0,202.0,202


### Análise

* Os três maiores clusters de pessoas pobres são de homens não-brancos (negro, pardo, amarelo e indígena);
* A maioria dos homens não-brancos que são pobres não vivem em zonas metropolitanas;
* Os quatro maiores clusters de pessoas pobres são de pessoas não-brancas;
* Os dois maiores clusters de mulheres pobres vivem em regiões urbanas;
* O maior cluster de homens pobres e brancos não vive em região urbana, o segundo maior vive;
* As mulheres pobres e brancas constituem o 9º cluster, o menor deles para um conjunto de sexo/raça;
* A idade média não parece ser um fator determinante para caracterizar os clusters, já que ela é muito próxima na maioria deles.
---

### Identificando Clusters de Pessoas Não-Pobres

In [ ]:
X_nao_pobre = dados[dados.poverty == 0].drop(['education', 'work', 'work_permit', 'poverty'], axis=1)

In [ ]:
for i in [False, StandardScaler, MinMaxScaler]:
    melhor_cluster(X_nao_pobre, 80, i)


Escalonador: Nenhum
KMeans --> Clusters: 2.0 | Silhueta: 0.5769349787742604
DBSCAN --> Epsilon (ε): 0.1 | Silhueta: 0.9204544589264779



Escalonador: StandardScaler
KMeans --> Clusters: 25.0 | Silhueta: 0.5672834211646839
DBSCAN --> Epsilon (ε): 0.8 | Silhueta: 0.5312510589523571



Escalonador: MinMaxScaler
KMeans --> Clusters: 13.0 | Silhueta: 0.8034478551862348
DBSCAN --> Epsilon (ε): 0.2 | Silhueta: 0.8137326826249022




#### O modelo com o melhor coeficiente de silhueta foi o DBSCAN com ε = 0.1 para dados não escalonados, analisemos os clusters encontrados:

In [ ]:
dbscan = DBSCAN(eps=0.1)
clusters(X_nao_pobre, dbscan)

,woman,mean_age,metropolitan_area,non_white,urban,size
cluster,,,,,,
-1,259,51.6,298,307.0,190,530
37,0,40.0,0,0.0,87,87
258,0,38.0,0,0.0,75,75
387,0,39.0,0,0.0,69,69
85,0,41.0,0,66.0,66,66
...,...,...,...,...,...,...
558,0,71.0,0,0.0,5,5
568,5,60.0,0,0.0,0,5
569,5,71.0,0,0.0,5,5


#### Embora a métrica de desempenho seja boa, percebe-se que o modelo não conseguiu de fato delimitar clusters significamente grandes, sendo o maior agrupamento (-1) o de anomalias. Anomalias são dados que não se enquadram em nenhum cluster. Avaliaremos então o modelo DBSCAN com ε = 0.2 e dados normalizados.

In [ ]:
dbscan = DBSCAN(eps=0.4)
clusters(X_nao_pobre, dbscan, MinMaxScaler)

,woman,mean_age,metropolitan_area,non_white,urban,size
cluster,,,,,,
6,0.0,41.336036,0.0,0.0,2220.0,2220
4,0.0,40.593512,0.0,1973.0,1973.0,1973
0,0.0,40.561267,1926.0,1926.0,1926.0,1926
5,1738.0,39.983314,0.0,0.0,1738.0,1738
1,0.0,42.578604,1533.0,0.0,1533.0,1533
2,1433.0,40.508723,1433.0,1433.0,1433.0,1433
3,1394.0,40.351506,0.0,1394.0,1394.0,1394
7,1313.0,40.952780,1313.0,0.0,1313.0,1313
11,0.0,44.595322,0.0,0.0,0.0,855


#### Agora sim conseguiu-se delimitar clusters grandes o suficiente e que permitam uma análise adequada.

### Análise

* O maior cluster de pessoas não-pobres são de homens brancos;
* Os três maiores clusters de pessoas não-pobres são de homens;
* Os oito maiores clusters de pessoas não-pobres vivem em regiões urbanas, e quatro deles especificamente em regiões metropolitanas;
* O cluster de mulheres não-brancas que vivem em regiões urbanas e metropolitanas é maior do que o de mulheres brancas na mesma condição;
* A idade novamente não parece ser um atributo importante para caracterizar os clusters.
---

#### Se você gostou do trabalho feito nesse notebook sinta-se à vontade para copiar qualquer parte do código e caso esteja vendo pelo Kaggle considere dar um like.